In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
import numpy as np
from astropy.table import Table
from numpy import random
import scipy
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic
from clmm import convert_units
import clmm
from clmm import GalaxyCluster, ClusterEnsemble, GCData
from clmm import Cosmology
from clmm.support import mock_data as mock

clmm.__version__

In [ ]:
cosmo = Cosmology(H0 = 71.0, Omega_dm0 = 0.265 - 0.0448, Omega_b0 = 0.0448, Omega_k0 = 0.0)

## Generating a cluster catalog and associated source catalogs
Below, we randomly generate the masses, redshifts, concentrations and coordinates of an ensemble of `n_clusters` clusters. For simplicity, the drawing is performed uniformly in logm and redshift (instead of following a halo mass function).


In [ ]:
# redshift and mass range of the galaxy clusters
z_bin = [0.2,0.25]
logm_bin = np.array([14, 14.1]) #Solar Mass

# number of clusters in the ensemble
n_clusters = 30

# random draw in the mass and redshift range (for simplicity, uniform instead of following an actual mass function)
cluster_m = 10**((logm_bin[1] - logm_bin[0])*np.random.random(n_clusters) + logm_bin[0]) #in M_sun
cluster_z =  (z_bin[1] - z_bin[0])*np.random.random(n_clusters) + z_bin[0] 

# random normal draw of cluster concentration, around c_mean
c_mean = 4.
lnc = abs(np.log(c_mean) + 0.01*np.random.randn(n_clusters))
concentration = np.exp(lnc)

# randomly draw cluster positions on the sky
ra = 0*(np.random.random(n_clusters)*(360 + 0) - 0) #from 0 to 360 deg
sindec = np.random.random(n_clusters)*(1 + 1) - 1
dec = 0*(np.arcsin(sindec)*180/np.pi) #from -90 to 90 deg

### Background galaxy catalog generation

For each cluster of the ensemble, we use `mock_data` to generate a background galaxy catalog and store the results in a `GalaxyCluster` object. The source redshifts follow the Chang et al. distribution and have associated pdfs. The shapes include shape noise and shape measurement errors. The cluster objects are then stored in `gclist`.

In [ ]:
gclist = []
# number of galaxies in each cluster field (alternatively, can use the galaxy density instead)
n_gals = 10000
#ngal_density = 10

for i in range(n_clusters):
    noisy_data_z = mock.generate_galaxy_catalog(cluster_m[i], cluster_z[i], concentration[i], cosmo, 
                                                cluster_ra=ra[i], cluster_dec=dec[i],
                                                zsrc = 'chang13', 
                                                delta_so=200, 
                                                massdef='critical',
                                                halo_profile_model='nfw', 
                                                zsrc_min=cluster_z[i] + 0.1,
                                                zsrc_max=3., 
                                                field_size=10., 
                                                shapenoise=.05, 
                                                photoz_sigma_unscaled=0.02, 
                                                ngals=n_gals,
                                                mean_e_err=.01)

    cl = clmm.GalaxyCluster('mock_cluster', ra[i], dec[i], cluster_z[i], noisy_data_z)

    cl.compute_tangential_and_cross_components(shape_component1='e1', shape_component2='e2', 
                                               tan_component='DS_t', cross_component='DS_x',
                                               cosmo=cosmo, is_deltasigma=True)

    cl.compute_galaxy_weights(z_source='ztrue', pzpdf='pzpdf', pzbins='pzbins', 
    shape_component1 = 'e1', shape_component2 = 'e2', 
    shape_component1_err = 'e_err', shape_component2_err = 'e_err', 
    use_photoz = True, use_shape_noise = True, use_shape_error = True, 
    weight_name = 'w_ls', cosmo = cosmo, is_deltasigma = True, add = True)
    gclist.append(cl)

## Creating ClusterEnsemble object
### Estimation of individual excess surface density profiles

In [ ]:
ensemble_id = 1
names = ['id', 'ra', 'dec', 'z', 'radius', 'gt', 'gx', 'W_l']
bins = np.logspace(np.log10(0.3),np.log10(5),10)
clusterensemble = ClusterEnsemble(ensemble_id, gclist, 
                                  tan_component_in='DS_t', cross_component_in='DS_x', 
                                  tan_component_out='DS_t', cross_component_out='DS_x', 
                                  weights_in = 'w_ls', weights_out = 'W_l', 
                                  bins=bins, bin_units='Mpc', cosmo=cosmo)

In [ ]:
clusterensemble.data.columns

### Stacked profile

In [ ]:
clusterensemble.make_stacked_radial_profile(tan_component='DS_t', cross_component='DS_x')

### Covariances (Bootstrap, sample, Jackknife)

In [ ]:
clusterensemble.compute_sample_covariance(tan_component='DS_t', cross_component='DS_x')
clusterensemble.compute_bootstrap_covariance(tan_component='DS_t', cross_component='DS_x', n_bootstrap=200)
#clusterensemble.compute_jackknife_covariance(n_side=4)

In [ ]:
from scipy.stats import binned_statistic
from clmm import convert_units

In [ ]:
plt.figure(figsize = (7,7))
plt.rcParams['axes.linewidth'] = 2
plt.plot(clusterensemble.data['radius'][0], clusterensemble.sample_tangential_covariance.diagonal()**0.5/1e13,'--',c = 'royalblue', label = 'Sample', linewidth = 3)
plt.plot(clusterensemble.data['radius'][0], clusterensemble.bootstrap_tangential_covariance.diagonal()**0.5/1e13,'-s',c = 'g', label = 'Bootstrap', linewidth = 3, markersize = 10)
#plt.plot(clusterensemble.data['radius'][0], clusterensemble.jackknife_tangential_covariance.diagonal()**0.5/1e13,c = 'r', label = 'Jackknife', linewidth = 3)
plt.xlabel('R [Mpc]', fontsize =20)
plt.ylabel(r'$\sigma_{\Delta\Sigma}\ (\times 10^{13} M_\odot /Mpc^2)$', fontsize =25)
plt.tick_params(axis='both', which = 'major', labelsize= 18)
plt.legend(frameon = False, fontsize = 20)
plt.loglog()
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/sprint_week_cov.png', bbox_inches='tight', dpi=300)

In [ ]:
fig, ax = plt.subplots(1,3, figsize =(20, 5))
plt.rcParams['axes.linewidth'] = 2
fig.subplots_adjust(wspace=0.15, hspace=0)
cov = [clusterensemble.sample_tangential_covariance, 
       clusterensemble.bootstrap_tangential_covariance,]
      #clusterensemble.jackknife_tangential_covariance,]
maximum = max(clusterensemble.sample_tangential_covariance.flatten())
label = ['Stack : Sample', 'Stack : Bootstrap', 'Stack : Jackknife']
for i in range(2):
    ax[i].set_title(label[i], fontsize = 20)
    ax[i].set_xlabel('radial bin index', fontsize = 18)
    ax[i].set_ylabel('radial bin index', fontsize = 18)
    ax[i].tick_params(axis='both', which = 'major', labelsize= 12)
    im = ax[i].imshow(cov[i], cmap = 'Reds',vmin = 0, vmax = maximum, origin = 'lower')
    plt.colorbar(im, ax=ax[i])
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/sprint_week_cov_full.png', bbox_inches='tight', dpi=300)

## Visualizing the stacked profiles and corresponding model

In [ ]:
moo = clmm.Modeling(massdef = 'critical', delta_mdef = 200, halo_profile_model = 'nfw')
moo.set_cosmo(cosmo)
# Average values of mass and concentration of the ensemble to be used below to overplot the model on the stacked profile
moo.set_concentration(concentration.mean())
moo.set_mass(cluster_m.mean())

In [ ]:
r_stack, gt_stack, gx_stack = (clusterensemble.stacked_data[c] for c in ('radius', 'DS_t', 'DS_x'))
plt.rcParams['axes.linewidth'] = 2
fig, axs = plt.subplots(1,2, figsize = (15,6), sharex = True)
fig.subplots_adjust(wspace=0.15, hspace=0)

err_gt = clusterensemble.sample_tangential_covariance.diagonal()**0.5/1e13
err_gx = clusterensemble.sample_cross_covariance.diagonal()**0.5/1e13

axs[0].errorbar(r_stack,gt_stack/1e13, err_gt,markersize = 5, c = 'r', fmt = 'o', capsize = 10, elinewidth = 1, zorder = 1000, alpha = 1, label = 'Stack')
axs[1].errorbar(r_stack,gx_stack/1e13, err_gx,markersize = 5, c = 'r', fmt = 'o', capsize = 10, elinewidth = 1, zorder = 1000, alpha = 1, label = 'Stack')

axs[0].plot(clusterensemble.data['radius'][0],moo.eval_excess_surface_density(clusterensemble.data['radius'][0], cluster_z.mean())/1e13, '--k', linewidth = 3, label = 'NFW 1-halo term', zorder = 100)
axs[1].plot(clusterensemble.data['radius'][0],0*moo.eval_excess_surface_density(clusterensemble.data['radius'][0], cluster_z.mean())/1e13, '--k', linewidth = 3, label = 'y=0', zorder = 100)

axs[0].set_xscale('log')

for i in range(n_clusters):
    axs[0].plot(clusterensemble.data['radius'][i],clusterensemble.data['DS_t'][i]/1e13, 'cyan', label = 'Individual', alpha = 1, linewidth = 1)
    axs[1].plot(clusterensemble.data['radius'][i],clusterensemble.data['DS_x'][i]/1e13, 'cyan', label = 'Individual', alpha = 1, linewidth = 1)
    if i == 0:
        axs[0].legend(frameon = False, fontsize = 20)
        axs[1].legend(frameon = False, fontsize = 20)
#axs[0].plot(np.average(clusterensemble.data['radius'], axis = 0), np.average(clusterensemble.data['gt'], weights = None, axis = 0)/1e13)
axs[0].set_xlabel('R [Mpc]', fontsize =20)
axs[1].set_xlabel('R [Mpc]', fontsize =20)
axs[0].tick_params(axis='both', which = 'major', labelsize= 18)
axs[1].tick_params(axis='both', which = 'major', labelsize= 18)
axs[0].set_title(r'$\Delta\Sigma_+$ $(\times 10^{13} M_\odot /Mpc^2)$', fontsize= 20)
axs[1].set_title(r'$\Delta\Sigma_x$  $(\times 10^{13} M_\odot /Mpc^2)$', fontsize= 20)
plt.show()
#axs[0].set_ylim(-.5,9)
#axs[1].set_ylim(-2,2)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/sprint_week_stack.png', bbox_inches='tight', dpi=300)